In [ ]:
import sys
import numpy as np
import xarray as xr
import pyproj
import cmocean as cmo
import warnings
import matplotlib as mpl
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

sys.path.append("..")
warnings.filterwarnings("ignore")

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

from PICO import PicoModel
from Plume import PlumeModel
from PICOP import PicoPlumeModel
from Simple import SimpleModels
from forcing import Forcing
from plotfunctions import *
from real_geometry import RealGeometry

In [ ]:
mpl.rcParams['figure.subplot.bottom'] = .15
mpl.rcParams['figure.subplot.top']    = .99
mpl.rcParams['figure.subplot.left']   = .01
mpl.rcParams['figure.subplot.right']  = .99
mpl.rcParams['figure.subplot.wspace'] = .02
mpl.rcParams['figure.subplot.hspace'] = .01

In [ ]:
"""Boundaries MITgcm"""
y1,y2 = {},{}
x1,x2 = {},{}
x1['CrossDots']  = 245
x2['CrossDots']  = 251
y1['CrossDots']  = -75.45
y2['CrossDots']  = -74.05
x1['Thwaites_e'] = 251
x2['Thwaites_e'] = 257
y1['Thwaites_e'] = -75.45
y2['Thwaites_e'] = -74.05
x1['PineIsland'] = 257
x2['PineIsland'] = 262
y1['PineIsland'] = -75.45
y2['PineIsland'] = -74.05
x1['Getz']       = 225
x2['Getz']       = 245.3
y1['Getz']       = -75.45
y2['Getz']       = -73.2

"""Locations for text"""
plon,plat = {},{}
plon['CrossDots']  = 250.5
plat['CrossDots']  = -74.4
plon['Thwaites_e'] = 253.5
plat['Thwaites_e'] = -74.6
plon['PineIsland'] = 257
plat['PineIsland'] = -74.8
plon['Getz']       = 231
plat['Getz']       = -74

In [ ]:
spinup = {}
spinup['PineIsland'] = 180
spinup['Thwaites_e'] = 60
spinup['CrossDots']  = 180
spinup['Getz']       = 60

date = '2021_02_25/'

In [ ]:
panelheight = .2
step = .125
figheight = 11

proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=245-360)

def addaxis(n,title):
    pos = 1-panelheight-n*step
    ax = fig.add_axes([.01,pos,.98,panelheight],projection=proj)
    ds = xr.open_dataset('../../data/BedMachine/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3000,4500),y=slice(7700,9500))
    ds = add_lonlat(ds)
    mask = xr.where(ds.mask==0,4,ds.mask)
    ax.contour(ds.lon,ds.lat,mask,[2.5],c='k',linewidths=.2,transform=ccrs.PlateCarree())

    ds = xr.open_dataset('../../data/BedMachine/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3300,4000),y=slice(6000,7700))
    ds = add_lonlat(ds)
    mask = xr.where(ds.mask==0,4,ds.mask)
    ax.contour(ds.lon,ds.lat,mask,[2.5],c='k',linewidths=.2,transform=ccrs.PlateCarree())
    ax.set_extent([227,259,-75.2,-73.1],crs=ccrs.PlateCarree())
    ax.set_title(title,y=.5,loc='left')
    ax.axis('off')
    return ax

def printvals(ax,geom,melt):
    ax.text(plon[geom],plat[geom],f'{np.nanmean(melt):.1f}',transform=ccrs.PlateCarree(),c='k',ha='center')


In [ ]:
%%time
for quant in ['average', 'sensitivity']:
    if quant=='average':
        timep = slice("1989-1-1","2018-12-31")
    elif quant=='sensitivity':
        timep_w = slice("2006-1-1","2012-1-1")
        timep_c = slice("2013-1-1","2017-1-1")

    fig = plt.figure(figsize=(8,figheight))

    """Geometry"""
    ax = addaxis(0,'a) Geometry')

    ds = xr.open_dataset('../../data/BedMachine/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3000,4500),y=slice(7700,9500))
    ds = add_lonlat(ds)
    draft = xr.where(ds.mask==3,(ds.surface-ds.thickness).astype('float64'),np.nan)
    draft = xr.where((ds.lon>-105).all() and (ds.lat>-73.7).all(),np.nan,draft)
    IM = plt.pcolormesh(ds.lon,ds.lat,draft,vmin=-1000,vmax=0,cmap=plt.get_cmap('cmo.rain_r'),transform=ccrs.PlateCarree(),shading='auto')
    ds = xr.open_dataset('../../data/BedMachine/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(3300,4000),y=slice(6000,7700))
    ds = add_lonlat(ds)
    draft = xr.where(ds.mask==3,(ds.surface-ds.thickness).astype('float64'),np.nan)
    draft = xr.where((ds.lon>-105).all() and (ds.lat>-73.7).all(),np.nan,draft)
    IM = plt.pcolormesh(ds.lon,ds.lat,draft,vmin=-1000,vmax=0,cmap=plt.get_cmap('cmo.rain_r'),transform=ccrs.PlateCarree(),shading='auto')

    '''Colorbar'''
    cax = fig.add_axes([.1,.05,.35,.015])
    cbar = plt.colorbar(IM, cax=cax,extend='min',orientation='horizontal')
    cbar.set_label('Draft depth [m]', labelpad=0)

    for geom in ['PineIsland','Thwaites_e','CrossDots','Getz']:
        if geom in ['PineIsland','Thwaites']:
            Tdeep, ztclw, ztclc = 1.2, -400, -600
        elif geom in ['CrossDots','Getz']:
            Tdeep, ztclw, ztclc = 0.7, -400, -700
        dp = RealGeometry(geom,n=5).create()
        dp = add_lonlat(dp)
        dw = Forcing(dp).tanh2(ztcl=ztclw, Tdeep=Tdeep)
        dc = Forcing(dp).tanh2(ztcl=ztclc, Tdeep=Tdeep)
        
        for j, model in enumerate(['Simple','Plume', 'PICO', 'PICOP', 'Layer','MITgcm']):
            title = [r'b) M$_+$','c) Plume','d) PICO','e) PICOP','f) Layer','g) MITgcm'][j]
            ax = addaxis(j+1,title)
            if model=='Simple':
                ds = SimpleModels(dw).compute()
                melt_w = ds['Mp']
                melt_c = SimpleModels(dc).compute()['Mp']
            elif model=='Plume':
                ds = PlumeModel(dw).compute_plume()
                melt_w = ds['m']
                melt_c = PlumeModel(dc).compute_plume()['m']
            elif model=='PICO':
                ds = PicoModel(ds=dw).compute_pico()[1]
                melt_w = ds['melt']
                melt_c = PicoModel(ds=dc).compute_pico()[1]['melt']
            elif model=='PICOP':
                ds = PicoPlumeModel(ds=dw).compute_picop()[2]
                melt_w = ds['m']
                melt_c = PicoPlumeModel(ds=dc).compute_picop()[2]['m']
            elif model=='Layer':
                ds = xr.open_dataset(f'../../results/Layer/{date}{geom}_tanh2_Tdeep{Tdeep}_ztcl{ztclw}_{spinup[geom]:.3f}.nc')
                ds = add_lonlat(ds)
                melt_w = np.where(ds.mask==3,ds.melt,np.nan)
                ds = xr.open_dataset(f'../../results/Layer/{date}{geom}_tanh2_Tdeep{Tdeep}_ztcl{ztclc}_{spinup[geom]:.3f}.nc')
                ds = add_lonlat(ds)
                melt_c = np.where(ds.mask==3,ds.melt,np.nan)
            elif model=='MITgcm':
                if quant=='average':
                    ds = xr.open_dataset('../../data/paulholland/melt.nc')
                    ds = ds.sel(LONGITUDE=slice(x1[geom],x2[geom]),LATITUDE=slice(y1[geom],y2[geom]),TIME=timep)
                    ds = ds.mean(dim='TIME')
                    melt = xr.where(ds.melt==0,np.nan,ds.melt)
                    melt_w, melt_c = melt, melt  # to make it compatible with the other output
                elif quant=='sensitivity':
                    ds = xr.open_dataset('../../data/paulholland/melt.nc')
                    ds = ds.sel(LONGITUDE=slice(x1[geom],x2[geom]),LATITUDE=slice(y1[geom],y2[geom]),TIME=timep_w)
                    ds = ds.mean(dim='TIME')
                    melt_w = xr.where(ds.melt==0,np.nan,ds.melt)
                    ds = xr.open_dataset('../../data/paulholland/melt.nc')
                    ds = ds.sel(LONGITUDE=slice(x1[geom],x2[geom]),LATITUDE=slice(y1[geom],y2[geom]),TIME=timep_c)
                    ds = ds.mean(dim='TIME')  
                    melt_c = xr.where(ds.melt==0,np.nan,ds.melt)
                ds['lon'] = ds.LONGITUDE
                ds['lat'] = ds.LATITUDE-.05
        
            if quant=='average':        melt = (melt_w+melt_c)/2
            elif quant=='sensitivity':  melt = melt_w-melt_c
                
            printvals(ax,geom,melt)
            melt = np.where(melt<1,1,melt)
            if quant=='average':        IM = plotmelt(ax,ds.lon,ds.lat,melt)
            elif quant=='sensitivity':  IM = plotdiffmelt(ax,ds.lon,ds.lat,melt)

    '''Colorbar'''
    cax = fig.add_axes([.5,.05,.35,.015])
    cbar = plt.colorbar(IM, cax=cax,extend='both',orientation='horizontal')
    if quant=='average':
        cbar.set_ticks([1,10,100])
        cbar.set_ticklabels([1,10,100])
        cbar.set_label('Melt rate [m/yr]', labelpad=0)
    elif quant=='sensitivity':    
        cbar.set_label('Melt rate difference [m/yr]', labelpad=0)
        
    """Save figure"""
    plt.savefig(f"../../figures/Amundsen_{quant}.png",dpi=300)